In [1]:
import numpy as np 
import pandas as pd
from sklearn import neighbors
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [2]:
# read train csv
prop_1_df = pd.read_csv("./Resources/properties_2016.csv")
prop_2_df = pd.read_csv("./Resources/properties_2017.csv")
prop_list = [prop_1_df, prop_2_df]

/local/home/ningzesun/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/local/home/ningzesun/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# hashottuborspa is nan means no spa
for total_df in prop_list:
    index = total_df.hashottuborspa.isnull()
    total_df.loc[index,'hashottuborspa'] = "None"
    #Assume if the pooltype id is null then pool/hottub doesnt exist 
    index = total_df.pooltypeid2.isnull()
    total_df.loc[index,'pooltypeid2'] = 0
    # whether has the pool
    index = total_df.poolcnt.isnull()
    total_df.loc[index,'poolcnt'] = 0
    # pool size are same so delete it.
    # fireplacecnt's nan is no fireplace
    index = total_df.fireplacecnt.isnull()
    total_df.loc[index,'fireplacecnt'] = 0
    # taxdelinquencyflag's nan is no taxdelinquencyflag
    index = total_df.taxdelinquencyflag.isnull()
    total_df.loc[index,'taxdelinquencyflag'] = 0
    #Assume if Null in garage count it means there are no garages
    index = total_df.garagecarcnt.isnull()
    total_df.loc[index,'garagecarcnt'] = 0

    #Likewise no garage means the size is 0 by default
    index = total_df.garagetotalsqft.isnull()
    total_df.loc[index,'garagetotalsqft'] = 0

    # nan in threequarterbathnbr means 0 by default
    index = total_df.threequarterbathnbr.isnull()
    total_df.loc[index,'threequarterbathnbr'] = 0

    # basementsqft nan mean 0
    index = total_df.basementsqft.isnull()
    total_df.loc[index,'basementsqft'] = 0

In [4]:
for i in prop_list:
    i.loc[:,'latitude'] = i.loc[:,'latitude']/1e6
    i.loc[:,'longitude'] = i.loc[:,'longitude']/1e6

In [5]:
prop_un_1_df = prop_1_df.loc[prop_1_df['latitude'].isna(),:]
prop_un_2_df = prop_2_df.loc[prop_2_df['latitude'].isna(),:]
prop_1_df = prop_1_df.loc[prop_1_df['latitude'].notna(),:]
prop_2_df = prop_2_df.loc[prop_2_df['latitude'].notna(),:]

In [6]:
from sklearn import neighbors
from sklearn.preprocessing import OneHotEncoder

def zoningcode2int( df, target ):
    storenull = df[ target ].isnull()
    enc = LabelEncoder( )
    df[ target ] = df[ target ].astype( str )

    print('fit and transform')
    df[ target ]= enc.fit_transform( df[ target ].values )
    print( 'num of categories: ', enc.classes_.shape  )
    df.loc[ storenull, target ] = np.nan
    print('recover the nan value')
    return enc

def fillna_knn( df, base, target, fraction = 0.1, threshold = 10, n_neighbors = 5 ):
    assert isinstance( base , list ) or isinstance( base , np.ndarray ) and isinstance( target, str ) 
    whole = [ target ] + base
    
    miss = df[target].isnull()
    notmiss = ~miss 
    nummiss = miss.sum()
    
    enc = OneHotEncoder()
    X_target = df.loc[ notmiss, whole ].sample( frac = fraction )
    
    enc.fit( X_target[ target ].unique().reshape( (-1,1) ) )
    
    Y = enc.transform( X_target[ target ].values.reshape((-1,1)) ).toarray()
    X = X_target[ base  ]
    
    print( 'fitting' )
    n_neighbors = n_neighbors
    clf = neighbors.KNeighborsClassifier( n_neighbors, weights = 'uniform' )
    clf.fit( X, Y )
    
    print( 'the shape of active features: ' ,enc.active_features_.shape )
    
    print( 'predicting' )
    Z = clf.predict(df.loc[miss, base])
    
    numunperdicted = Z[:,0].sum()
    if numunperdicted / nummiss *100 < threshold :
        print( 'writing result to df' )    
        df.loc[ miss, target ]  = np.dot( Z , enc.active_features_ )
        print( 'num of unperdictable data: ', numunperdicted )
        return enc
    else:
        print( 'out of threshold: {}% > {}%'.format( numunperdicted / nummiss *100 , threshold ) )

In [7]:
geo_col = ['buildingqualitytypeid', 'regionidcity', 'regionidneighborhood', 
           'regionidzip', 'unitcnt', 'yearbuilt', 'lotsizesquarefeet']
geo_trans = ['propertycountylandusecode', 'propertyzoningdesc']

In [10]:
import warnings
warnings.filterwarnings('ignore')
fillna_knn(df = prop_1_df, base = ['latitude', 'longitude'], target = 'buildingqualitytypeid', fraction = 0.15)
fillna_knn(df = prop_2_df, base = ['latitude', 'longitude'], target = 'buildingqualitytypeid', fraction = 0.15)

fitting
the shape of active features:  (12,)
predicting
writing result to df
num of unperdictable data:  27650.0
fitting
the shape of active features:  (12,)
predicting
writing result to df
num of unperdictable data:  55.0


OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [11]:
prop_1_df = pd.concat([prop_1_df, prop_un_1_df])
prop_2_df = pd.concat([prop_2_df, prop_un_2_df])
prop_1_df[['parcelid', 'buildingqualitytypeid']].to_csv("./Resources/properties_buildingqualitytypeid_2016.csv")
prop_2_df[['parcelid', 'buildingqualitytypeid']].to_csv("./Resources/properties_buildingqualitytypeid_2017.csv")